In [2]:
import pandas as pd
import numpy as np
import os

Definições Iniciais

In [3]:
#path dos arquivos
years = [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
path_option = {}
for j in years:
    var = 'input/' + str(j) + '/dados/'
    data = 'microdados_ed_basica_' + str(j) + '.csv'
    path = os.path.abspath(os.path.join('..', var))
    path_option[j] = os.path.join(path, data)

In [4]:
#variáveis do dicionário que não vão ser adicionadas agora
variable_exclude = ['etapa_ensino_creche_comum',
'etapa_ensino_pre_escola_comum',
'etapa_ensino_fundamental_anos_iniciais_comum',
'etapa_ensino_fundamental_anos_finais_comum',
'etapa_ensino_medio_comum',
'etapa_ensino_medio_integrado_comum',
'etapa_ensino_medio_normal_comum',
'etapa_ensino_profissional_comum',
'etapa_ensino_eja_fundamental_comum',
'etapa_ensino_eja_medio_comum',
'etapa_ensino_eja_profissional_comum',
'etapa_ensino_creche_especial_exclusiva',
'etapa_ensino_pre_escola_especial_exclusiva',
'etapa_ensino_fundamental_anos_iniciais_especial_exclusiva',
'etapa_ensino_fundamental_anos_finais_especial_exclusiva',
'etapa_ensino_medio_especial_exclusiva',
'etapa_ensino_medio_integrado_especial_exclusiva',
'etapa_ensino_medio_normal_especial_exclusiva',
'etapa_ensino_profissional_especial_exclusiva',
'etapa_ensino_eja_fundamental_especial_exclusiva',
'etapa_ensino_eja_medio_especial_exclusiva']

In [5]:
#variáveis tipo bool, tirar 9
treat_variable_bool = ['tratamento_lixo_separacao',
'tratamento_lixo_reutilizacao',
'tratamento_lixo_reciclagem',
'tratamento_lixo_inexistente',
'acesso_internet_computador',
'acesso_internet_dispositivo_pessoal',
'organizacao_serie_ano',
'organizacao_semestre',
'organizacao_fundamental_ciclo',
'organizacao_grupo_nao_seriado',
'organizacao_modulo',
'organizacao_alternancia',
'exame_selecao',
'redes_sociais',
'espaco_atividade_comunidade',
'espaco_equipamento_alunos']

In [6]:
#variáveis tipo date: formatar data
treat_variable_date = ['data_ano_letivo_inicio', 'data_ano_letivo_termino']

In [7]:
#dicionário para corrigir data
month_inep_number = {'JAN': '01', 'FEB':'02', 'MAR': '03', 'APR': '04', 'MAY': '05', 'JUN': '06', 
'JUL': '07', 'AUG': '08', 'SEP': '09', 'OCT': '10', 'NOV': '11', 'DEC': '12'}

In [8]:
#arquitetura: renomear e ordernar
arquitetura = pd.read_csv(r'C:\Users\ingri\OneDrive - 033ms\workspace\br_inep_censo_escolar\extra\architecture\escola\arquitetura.csv', nrows=373, na_filter = False)
original_name_variable = arquitetura.original_name_2021.tolist() #variáveis no novo censo escolar
original_name_variable = list(filter(None, original_name_variable))
new_name_variable_aux = arquitetura.name.tolist()
new_name_variable = [ele for ele in new_name_variable_aux if ele not in variable_exclude]
rename = dict(zip(original_name_variable, new_name_variable))
order = new_name_variable

Tratamento

In [ ]:
#tratamento inicial
list_df = []
for i in range(len(years)):
    df_aux = pd.DataFrame()
    ch = 0 
    for chunk in pd.read_csv(path_option[years[i]], chunksize = 20000, na_filter = False, sep = ';', usecols = original_name_variable, engine = 'python', dtype='string'):
        ch = ch + 1
        print("fazendo chunck {} de {} ".format(ch,years[i]))
        chunk.rename(columns = rename, inplace = True)
        chunk = chunk[order]
        for j in treat_variable_date:
            aux = chunk[j]
            chunk[j] = aux.str[5:9] + '-' + aux.str[2:5] + '-' + aux.str[:2]
            chunk[j].replace({'--':''}, inplace = True)
            chunk[j] = chunk[j].str.replace('|'.join(month_inep_number), lambda string: month_inep_number[string.group()])
            for f in treat_variable_bool:
                chunk[f].replace({'9':''}, inplace = True)
        df_aux = pd.concat([df_aux, chunk])
    list_df.append(df_aux)

Output

In [19]:
def path_out ():
    partic = 'output\escola'
    partic_aux = "\\ano={}\\sigla_uf={}"
    partic = os.path.abspath(os.path.join('..', partic))
    if (not os.path.isdir(partic)):
        os.mkdir(partic)
    path= os.path.join(partic + partic_aux)
    return path

In [20]:
base = list_df.copy()
ufs = list(base[0]['sigla_uf'].unique())
for i in range(len(years)):
    mg_a = base[i]
    for uf in ufs:
        path_i = path_out().format(years[i], uf)
        if (not os.path.isdir(path_i)):
            os.makedirs(path_i)
        mg_auf = mg_a[mg_a['sigla_uf'] == uf]
        mg_auf.drop(['ano','sigla_uf'], axis=1, inplace=True)
        mg_auf.to_csv(path_i + '\escola.csv', index = False)
        del path_i